---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [58]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [106]:
def separateStateRegionValues(x):
    global curStateName
    
    #print(x[0])
    if (x[0].endswith('[edit]')):
        curStateName = x[0].replace('[edit]','')
        return np.NAN
    else:
        regionName = x[0].split(' (')[0]
        #print({'State': curStateName, 'RegionName': regionName})
        return {'State': curStateName, 'RegionName': regionName}

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df = pd.read_csv('university_towns.txt', sep='\n', header=None)
                                 
    
    return (df.apply(separateStateRegionValues, axis=1)
             .dropna()
             .apply(pd.Series))[['State','RegionName']]
                              

In [107]:
get_list_of_university_towns()

State                    RegionName
1          Alabama                        Auburn
2          Alabama                      Florence
3          Alabama                  Jacksonville
4          Alabama                    Livingston
5          Alabama                    Montevallo
6          Alabama                          Troy
7          Alabama                    Tuscaloosa
8          Alabama                      Tuskegee
10          Alaska                     Fairbanks
12         Arizona                     Flagstaff
13         Arizona                         Tempe
14         Arizona                        Tucson
16        Arkansas                   Arkadelphia
17        Arkansas                        Conway
18        Arkansas                  Fayetteville
19        Arkansas                     Jonesboro
20        Arkansas                      Magnolia
21        Arkansas                    Monticello
22        Arkansas                  Russellville
23        Arkansas                        Searcy
25      California                        Angwin
26      California                        Arcata
27      California                      Berkeley
28      California                         Chico
29      California                     Claremont
30      California                        Cotati
31      California                         Davis
32      California                        Irvine
33      California                    Isla Vista
34      California  University Park, Los Angeles
..             ...                           ...
533       Virginia                          Wise
534       Virginia                    Chesapeake
536     Washington                    Bellingham
537     Washington                        Cheney
538     Washington                    Ellensburg
539     Washington                       Pullman
540     Washington  University District, Seattle
542  West Virginia                        Athens
543  West Virginia                    Buckhannon
544  West Virginia                      Fairmont
545  West Virginia                     Glenville
546  West Virginia                    Huntington
547  West Virginia                    Montgomery
548  West Virginia                    Morgantown
549  West Virginia                 Shepherdstown
550  West Virginia                  West Liberty
552      Wisconsin                      Appleton
553      Wisconsin                    Eau Claire
554      Wisconsin                     Green Bay
555      Wisconsin                     La Crosse
556      Wisconsin                       Madison
557      Wisconsin                     Menomonie
558      Wisconsin                     Milwaukee
559      Wisconsin                       Oshkosh
560      Wisconsin                   Platteville
561      Wisconsin                   River Falls
562      Wisconsin                 Stevens Point
563      Wisconsin                      Waukesha
564      Wisconsin                    Whitewater
566        Wyoming                       Laramie

[517 rows x 2 columns]

In [63]:
def get_recession():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=219, usecols='E,G', names=['Quarter','GDP'])
    
    recession = False
    dfr=[]

    for index, row in df.iterrows():
        if (index == 0 or index == (df.shape[0]-1)):
            continue
        if (recession == False and(df.iloc[index-1]['GDP'] > df.iloc[index]['GDP'] > df.iloc[index+1]['GDP'])):
            recession = True
            rstart = df.iloc[index]
            #rstartIndex = index
            #print("rstart" + rstart)
        elif (recession == True and (df.iloc[index-1]['GDP'] < df.iloc[index]['GDP'] < df.iloc[index+1]['GDP'])):
            recession = False
            rend = df.iloc[index]
            #rendindex = index
            dfr.append((rstart, rend))
            #print("rend:" + rend)
    return dfr

In [64]:
get_recession()

[(Quarter     2008q3
  GDP        14891.6
  Name: 34, dtype: object, Quarter     2009q3
  GDP        14402.5
  Name: 38, dtype: object)]

In [65]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return get_recession()[0][0].Quarter
        
get_recession_start()

'2008q3'

In [66]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return get_recession()[0][1].Quarter

get_recession_end()

'2009q3'

In [68]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=219, usecols='E,G', names=['Quarter','GDP'])
    r = get_recession()[0]
    start, end = r[0].name, r[1].name
    #print(start, end)
    
    rData = df.iloc[start:end+1]
    return rData.loc[rData['GDP'] == np.min(rData['GDP'])].iloc[0]['Quarter']

get_recession_bottom()

'2009q2'

In [111]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    hdf = pd.read_csv('City_Zhvi_AllHomes.csv')
    hdf.State = hdf.State.apply(lambda x: states[x])
    #Separate text columns
    hds = hdf[['RegionName','State']]
    
    #Transform 
    hdf = hdf.T
    
    #Remove text columns
    hdf2 = hdf.drop(['RegionID','RegionName','State','Metro','CountyName','SizeRank'])
    
    #Set column types to Number
    hdf2 = hdf2.apply(pd.to_numeric, downcast='integer')
    
    #Dict to get quarter for given month
    qmap = {'01': 'q1', '02': 'q1', '03': 'q1', '04': 'q2', '05': 'q2', '06': 'q2', '07': 'q3', '08': 'q3', '09': 'q3', '10': 'q4', '11': 'q4', '12': 'q4'}
    
    #Get year
    hdf2['year'] = hdf2.index.str.slice(0,4)
    #Get month
    hdf2['q'] = hdf2.index.str.slice(5,7)
    
    #Change to month's Quarter 
    hdf2.q = hdf2.q.apply(lambda x: qmap[x])
    
    #Combine year and quarter
    hdf2['yearq'] = hdf2.year + hdf2.q
    
    #Remove years before 2000
    hdf2 = hdf2[pd.to_numeric(hdf2['year']) >= 2000]
    
    #Set year-quarter as index. When transformed index labels will become column labels.
    hdf2 = hdf2.set_index('yearq')

    #Group by year/quarter and compute mean. Then transform so year/quarter becomes columns labels
    hdf3 = hdf2.groupby('yearq').mean().T
    
    #Merge with State/Region dataframe to get final dataset
    return pd.merge(hds, hdf3, how='inner', left_index=True, right_index=True).set_index(["State","RegionName"])


In [167]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #get house price 
    housePriceDF = convert_housing_data_to_quarters()[[get_recession_start(), get_recession_bottom()]]
    
    
    # Get university town housing data for recession start quarter
    mergeDF = pd.merge(housePriceDF1, get_list_of_university_towns(), left_on=['State','RegionName'], right_on=['State','RegionName'], how='left', indicator=True)
    
    mergeDF['priceratio'] = mergeDF[get_recession_start()]/mergeDF[get_recession_bottom()]
    
    # Get university town housing data for recession bottom quarter
    utownhouseprices = mergeDF[mergeDF._merge == 'both']
    
    houseprices = mergeDF[mergeDF._merge == 'left_only']
    
    # Run t-test
    statistic,pvalue = ttest_ind(utownhouseprices.priceratio, houseprices.priceratio, nan_policy='omit')
    
    return (pvalue<0.01, pvalue, "university town" if (utownhouseprices.priceratio.mean() < houseprices.priceratio.mean()) else "non-university town")

run_ttest()

(True, 0.005496427353633026, 'university town')